In [1]:
import os
from io import StringIO # if going with no saving csv file
import gcsfs
import dask.dataframe as dd
from dask.delayed import delayed
from pathlib import Path
import pandas as pd
import os

In [2]:
from pprint import pprint, pp
from IPython.display import display, Markdown
from icecream import ic

In [3]:
%%capture 

from nanoHUB.application import Application
application = Application.get_instance()

In [4]:
%%capture 

rfm_engine = application.new_db_engine('rfm_data')

In [5]:
sql = '''
INSERT INTO rfm_data.user_descriptors (id, username,name,email, lastvisitDate, registerDate)
SELECT id, username, name, email, lastvisitDate, registerDate
FROM nanohub.jos_users
'''

In [6]:
# with rfm_engine.connect() as connection:
#     with connection.begin():
#         connection.execute(sql)

In [21]:
sql = '''
INSERT INTO rfm_data.user_descriptors (id, username,name,email, lastvisitDate, registerDate, user_lifetime)
SELECT id, username, name, email, lastvisitDate, registerDate, timestampdiff(DAY, registerDate, lastvisitDate)
FROM nanohub.jos_users
WHERE NOT EXISTS (SELECT 1 FROM rfm_data.user_descriptors WHERE rfm_data.user_descriptors.id = nanohub.jos_users.id)
'''

In [22]:
with rfm_engine.connect() as connection:
    with connection.begin():
        connection.execute(sql)

In [20]:
sql = '''
UPDATE rfm_data.user_descriptors INNER JOIN nanohub.jos_users
    ON rfm_data.user_descriptors.id = nanohub.jos_users.id
SET rfm_data.user_descriptors.user_lifetime = timestampdiff(DAY, nanohub.jos_users.registerDate, nanohub.jos_users.lastvisitDate)
WHERE rfm_data.user_descriptors.lastvisitDate != nanohub.jos_users.lastvisitDate
'''

In [15]:
with rfm_engine.connect() as connection:
    with connection.begin():
        connection.execute(sql)